<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/Interactive_Psychrometric_Chart_with_File_Selection_and_Dynamic_Columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import io # To simulate CSV file reading from a string
import os # Will be used for actual file system interaction (commented out for now)
import glob # Will be used for actual file system interaction (commented out for now)

# 1. 기본 상수 정의 (Constants)
C_PA = 1.006  # 건공기의 정압비열 (Specific heat of dry air) (kJ/kg°C)
C_PW = 1.86   # 수증기의 정압비열 (Specific heat of water vapor) (kJ/kg°C)
H_FG0 = 2501  # 0°C에서의 물의 증발잠열 (Latent heat of vaporization at 0°C) (kJ/kg)
P_ATM = 101.325 # 표준 대기압 (Standard atmospheric pressure) (kPa)

# 2. 습공기 상태량 계산 함수 (Psychrometric Calculation Functions) - Unchanged
def get_saturation_vapor_pressure_kPa(temp_c):
    if temp_c < -60: return 0.0
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

def get_actual_vapor_pressure_kPa(temp_c, rh_percent):
    p_ws = get_saturation_vapor_pressure_kPa(temp_c)
    return (rh_percent / 100.0) * p_ws

def get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    p_w = get_actual_vapor_pressure_kPa(temp_c, rh_percent)
    denominator = p_atm_kPa - p_w
    if denominator <= 1e-9:
        if abs(p_w - get_saturation_vapor_pressure_kPa(temp_c)) < 1e-6 and rh_percent > 99.99:
             p_w_adjusted = get_saturation_vapor_pressure_kPa(temp_c) * 0.99999
             denominator_adjusted = p_atm_kPa - p_w_adjusted
             if denominator_adjusted <= 1e-9: return float('inf')
             return 0.621945 * p_w_adjusted / denominator_adjusted
        else:
            return float('inf')
    return 0.621945 * p_w / denominator

def get_enthalpy_kJ_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    w = get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa)
    if w == float('inf'): return float('inf')
    return C_PA * temp_c + w * (H_FG0 + C_PW * temp_c)

def get_temp_from_enthalpy_humidity_ratio(h_kJ_kg, w_kg_kg):
    denominator = C_PA + w_kg_kg * C_PW
    if abs(denominator) < 1e-9: return float('nan')
    return (h_kJ_kg - w_kg_kg * H_FG0) / denominator

# 3. Plotly용 습공기선도 배경 그리기 함수 - Unchanged from previous Plotly version
def add_psychrometric_background_to_fig(fig, temp_range_c, w_range_kg_kg):
    temps_c_plot = np.linspace(temp_range_c[0], temp_range_c[1], 100)
    w_saturated = np.array([get_humidity_ratio_kg_kg(t, 100) for t in temps_c_plot])
    valid_sat_indices = ~np.isinf(w_saturated) & (w_saturated <= w_range_kg_kg[1])
    fig.add_trace(go.Scatter(
        x=temps_c_plot[valid_sat_indices], y=w_saturated[valid_sat_indices],
        mode='lines', name='RH 100%', line=dict(color='black', width=1.5),
        hoverinfo='skip', showlegend=True
    ))
    rh_curves_to_plot = [20, 40, 60, 80]
    for rh_val in rh_curves_to_plot: # Renamed rh to rh_val to avoid conflict
        w_rh_plot = np.array([get_humidity_ratio_kg_kg(t, rh_val) for t in temps_c_plot]) # Renamed w_rh to w_rh_plot
        valid_rh_indices = ~np.isinf(w_rh_plot) & (w_rh_plot <= w_range_kg_kg[1]) & (w_rh_plot >= w_range_kg_kg[0])
        if np.any(valid_rh_indices):
            fig.add_trace(go.Scatter(
                x=temps_c_plot[valid_rh_indices], y=w_rh_plot[valid_rh_indices],
                mode='lines', name=f'RH {rh_val}%', line=dict(color='blue', width=0.8, dash='dash'),
                hoverinfo='skip', showlegend=True
            ))
    h_min_approx = get_enthalpy_kJ_kg(temp_range_c[0], 20 if temp_range_c[0] > 0 else 80)
    h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 100)
    if h_min_approx == float('inf'): h_min_approx = 0
    if h_max_approx == float('inf'): h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 80, P_ATM)
    if h_max_approx == float('inf'): h_max_approx = 120
    enthalpy_lines_values = np.unique(np.linspace(round(h_min_approx / 10) * 10, round(h_max_approx / 10) * 10, 10).astype(int))
    enthalpy_lines_values = [h for h in enthalpy_lines_values if h != float('inf') and h != float('-inf')]
    for h_val in enthalpy_lines_values:
        if np.isinf(h_val) or np.isnan(h_val): continue
        w_for_h_line = np.linspace(w_range_kg_kg[0], w_range_kg_kg[1], 50)
        temps_for_h_line = np.array([get_temp_from_enthalpy_humidity_ratio(h_val, w) for w in w_for_h_line])
        valid_h_points_indices = ~np.isnan(temps_for_h_line) & \
                                 (temps_for_h_line >= temp_range_c[0]) & \
                                 (temps_for_h_line <= temp_range_c[1]) & \
                                 (w_for_h_line >= w_range_kg_kg[0]) & \
                                 (w_for_h_line <= w_range_kg_kg[1])
        if np.sum(valid_h_points_indices) > 1:
            temps_plot_h = temps_for_h_line[valid_h_points_indices]
            w_plot_h = w_for_h_line[valid_h_points_indices]
            fig.add_trace(go.Scatter(
                x=temps_plot_h, y=w_plot_h, mode='lines',
                name=f'h={h_val}kJ/kg', line=dict(color='green', width=0.8, dash='dot'),
                hoverinfo='skip', showlegend=False # Hide individual enthalpy lines from legend
            ))
            if len(temps_plot_h) > 0:
                text_x, text_y = temps_plot_h[0], w_plot_h[0]
                if temp_range_c[0] <= text_x <= temp_range_c[1] and w_range_kg_kg[0] <= text_y <= w_range_kg_kg[1]:
                    fig.add_annotation(x=text_x, y=text_y, text=f"{h_val}", showarrow=False,
                                       font=dict(color="green", size=9), xanchor="right", yanchor="bottom", textangle=-30)

# --- New File and Column Handling Functions ---
def find_oac_csv_files_simulated(virtual_file_system):
    """Simulates finding CSV files with 'OAC' in their names."""
    print("Simulating file search...")
    oac_files = {name: content for name, content in virtual_file_system.items() if "oac" in name.lower() and name.endswith(".csv")}
    if not oac_files:
        print("No simulated 'OAC' CSV files found.")
        return {}
    return oac_files

def select_csv_file_simulated(oac_files_dict):
    """Prompts user to select a simulated file."""
    if not oac_files_dict:
        return None, None

    print("\nAvailable 'OAC' CSV files (simulated):")
    file_list = list(oac_files_dict.keys())
    for i, filename in enumerate(file_list):
        print(f"  {i+1}. {filename}")

    while True:
        try:
            choice = int(input(f"Select a file by number (1-{len(file_list)}): "))
            if 1 <= choice <= len(file_list):
                selected_filename = file_list[choice-1]
                return selected_filename, oac_files_dict[selected_filename]
            else:
                print("Invalid choice. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# Configuration for dynamic column mapping
STAGE_MAPPING_DEFINITIONS = [
    {'id': 'PHC', 'display_name': "1. PH/C", 'stage_keywords': ["PH/C", "PREHEAT"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#ff7f0e'},
    {'id': 'PCC', 'display_name': "2. PC/C", 'stage_keywords': ["PC/C", "PRECOOL"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#2ca02c'},
    {'id': 'CC',  'display_name': "3. C/C", 'stage_keywords': ["C/C", "COOLINGCOIL", "냉각코일", "COOL"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#d62728'},
    {'id': 'HC',  'display_name': "4. H/C", 'stage_keywords': ["H/C", "HEATINGCOIL", "재열코일", "승온", "HEAT"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#9467bd'}
]

def map_columns_from_csv(df_columns):
    """Dynamically maps CSV columns to stages based on keywords."""
    mapped_stages_config = []
    print("\nAttempting to map columns...")

    for stage_def in STAGE_MAPPING_DEFINITIONS:
        found_temp_col = None
        found_rh_col = None

        for col_name in df_columns:
            col_lower = col_name.lower()
            # Check for stage keyword
            if any(sk.lower() in col_lower for sk in stage_def['stage_keywords']):
                # Check for temp keyword
                if stage_def['temp_keyword'].lower() in col_lower:
                    if found_temp_col is None: # Take first match
                        found_temp_col = col_name
                    else:
                        print(f"  Warning: Multiple temperature columns found for {stage_def['display_name']}. Using '{found_temp_col}'. Found also '{col_name}'")

                # Check for RH keyword
                if stage_def['rh_keyword'].lower() in col_lower:
                    if found_rh_col is None: # Take first match
                        found_rh_col = col_name
                    else:
                        print(f"  Warning: Multiple humidity columns found for {stage_def['display_name']}. Using '{found_rh_col}'. Found also '{col_name}'")

        if found_temp_col and found_rh_col:
            mapped_stages_config.append({
                'name': stage_def['display_name'],
                'temp_col': found_temp_col,
                'rh_col': found_rh_col,
                'color': stage_def['color']
            })
            print(f"  Successfully mapped stage: {stage_def['display_name']}")
            print(f"    Temp Column: '{found_temp_col}'")
            print(f"    RH Column  : '{found_rh_col}'")
        else:
            print(f"  Could not fully map stage: {stage_def['display_name']}. Temp found: {found_temp_col}, RH found: {found_rh_col}")

    if not mapped_stages_config:
        print("Warning: No stages could be mapped from the CSV columns.")
    return mapped_stages_config

# 4. 대화형 차트 생성 함수 (Function to create interactive chart with Plotly)
def create_interactive_psychrometric_chart_plotly(df, mapped_stages_config, selected_filename):
    if df.empty:
        print("DataFrame is empty. Cannot create chart.")
        return
    if not mapped_stages_config:
        print("No mapped stages configuration. Cannot create chart.")
        return

    # Pre-calculate W and H for all points to determine chart bounds
    all_temps_for_bounds = []
    all_ws_for_bounds = []
    for index, row in df.iterrows():
        for stage in mapped_stages_config:
            try:
                temp = row[stage['temp_col']]
                rh = row[stage['rh_col']]
                all_temps_for_bounds.append(temp)
                w = get_humidity_ratio_kg_kg(temp, rh)
                if w != float('inf') and w < 0.1: # Cap w for bounds calculation
                    all_ws_for_bounds.append(w)
            except KeyError as e:
                print(f"Warning: Column {e} not found in DataFrame for stage {stage['name']} during bounds calculation. Skipping.")
                continue # Skip this stage for this row if a column is missing

    if not all_temps_for_bounds or not all_ws_for_bounds:
        print("No valid data points for chart bounds after psychrometric calculations. Check column mapping and data.")
        # Fallback bounds
        min_temp_data, max_temp_data = 0, 40
        min_w_data, max_w_data = 0, 0.025
    else:
        min_temp_data, max_temp_data = min(all_temps_for_bounds), max(all_temps_for_bounds)
        min_w_data, max_w_data = min(all_ws_for_bounds), max(all_ws_for_bounds)


    temp_margin = (max_temp_data - min_temp_data) * 0.1 if (max_temp_data - min_temp_data) > 0 else 5
    w_margin = (max_w_data - min_w_data) * 0.1 if (max_w_data - min_w_data) > 0 else 0.002

    chart_min_temp = min_temp_data - temp_margin
    chart_max_temp = max_temp_data + temp_margin * 1.5
    chart_min_w = 0
    chart_max_w = max_w_data + w_margin * 2.0

    w_sat_at_max_temp_chart = get_humidity_ratio_kg_kg(chart_max_temp, 100)
    if w_sat_at_max_temp_chart != float('inf'):
        chart_max_w = max(chart_max_w, w_sat_at_max_temp_chart * 1.05)
    chart_max_w = min(chart_max_w, 0.035)

    fig = go.Figure()
    add_psychrometric_background_to_fig(fig, (chart_min_temp, chart_max_temp), (chart_min_w, chart_max_w))

    fig.add_trace(go.Scatter(
        x=[], y=[], mode='markers', marker=dict(size=10), name="Process Points", # Color will be set per point in frames
        text=[], hovertemplate="<b>%{text}</b><br>Temp: %{x:.1f}°C<br>W: %{y:.4f} kg/kg<br>RH: %{customdata[0]:.0f}%<br>H: %{customdata[1]:.1f} kJ/kg<extra></extra>",
        customdata=[], showlegend=False # Hide this generic legend item
    ))
    fig.add_trace(go.Scatter(
        x=[], y=[], mode='lines', line=dict(color='gray', width=1.5, dash='solid'), name="Process Path",
        hoverinfo='skip', showlegend=False # Hide this generic legend item
    ))

    frames = []
    for idx, row_data in df.iterrows():
        timestamp_str = row_data['Timestamp'].strftime('%Y-%m-%d %H:%M:%S')
        frame_points_x = []
        frame_points_y = []
        frame_points_text = []
        frame_points_customdata = []
        frame_points_colors = []

        current_annotations = []

        for i, stage_conf in enumerate(mapped_stages_config):
            try:
                temp_c = row_data[stage_conf['temp_col']]
                rh_p = row_data[stage_conf['rh_col']]
            except KeyError: # If a mapped column is somehow missing in a row
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_points_text.append(stage_conf['name'] + " (Data Error)")
                frame_points_customdata.append([None, None])
                frame_points_colors.append('grey')
                continue

            w = get_humidity_ratio_kg_kg(temp_c, rh_p)
            h = get_enthalpy_kJ_kg(temp_c, rh_p)

            if w != float('inf') and h != float('inf') and w < chart_max_w * 1.5 :
                frame_points_x.append(temp_c)
                frame_points_y.append(w)
                frame_points_text.append(stage_conf['name'])
                frame_points_customdata.append([rh_p, h])
                frame_points_colors.append(stage_conf['color'])

                current_annotations.append(dict(
                    x=temp_c, y=w,
                    text=f"<b>{stage_conf['name']}</b><br>{temp_c}°C, {rh_p}%<br>h={h:.1f}",
                    showarrow=True, arrowhead=1, arrowwidth=1, arrowcolor=stage_conf['color'],
                    ax=20, ay=-30,
                    font=dict(size=9, color='black'), # Annotation text color
                    bordercolor=stage_conf['color'], borderwidth=1,
                    bgcolor="rgba(255,255,255,0.8)"
                ))
            else:
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_points_text.append(stage_conf['name'] + " (Calc Error)")
                frame_points_customdata.append([None, None])
                frame_points_colors.append('grey')

        frames.append(go.Frame(
            name=timestamp_str,
            data=[
                go.Scatter(x=frame_points_x, y=frame_points_y, text=frame_points_text,
                           customdata=np.array(frame_points_customdata).T if frame_points_customdata and any(cd is not None for cd_list in frame_points_customdata for cd in cd_list) else [],
                           marker=dict(color=frame_points_colors, size=10, line=dict(width=1, color='DarkSlateGrey'))), # Update points trace with colors
                go.Scatter(x=frame_points_x, y=frame_points_y, line=dict(color='dimgray'))  # Update lines trace
            ],
            layout=go.Layout(
                annotations=current_annotations,
                title_text=f"Interactive Psychrometric Chart: {selected_filename} - {timestamp_str}"
            )
        ))

    fig.frames = frames

    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Time: ", "font": {"size": 14}},
        pad={"t": 50, "b":10},
        steps=[]
    )]

    for i, timestamp in enumerate(df['Timestamp']):
        ts_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        slider_step = dict(
            label=timestamp.strftime('%H:%M'),
            method="animate",
            args=[[ts_str], dict(mode="immediate",
                                 frame=dict(duration=100, redraw=True),
                                 transition=dict(duration=50))]
        )
        sliders[0]['steps'].append(slider_step)

    initial_timestamp_str = df['Timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M:%S')
    fig.update_layout(
        title=f"Interactive Psychrometric Chart: {selected_filename} - {initial_timestamp_str}",
        xaxis=dict(title='Dry Bulb Temperature (°C)', range=[chart_min_temp, chart_max_temp]),
        yaxis=dict(title='Humidity Ratio (kg_water / kg_dry_air)', range=[chart_min_w, chart_max_w]),
        sliders=sliders,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, traceorder="normal"),
        margin=dict(l=60, r=60, b=120, t=100),
        hovermode="closest"
    )

    if frames: # Initialize first frame
        first_frame = frames[0]
        fig.data[len(fig.data)-2].x = first_frame.data[0].x
        fig.data[len(fig.data)-2].y = first_frame.data[0].y
        fig.data[len(fig.data)-2].text = first_frame.data[0].text
        fig.data[len(fig.data)-2].customdata = first_frame.data[0].customdata
        if hasattr(first_frame.data[0], 'marker') and hasattr(first_frame.data[0].marker, 'color'):
             fig.data[len(fig.data)-2].marker.color = first_frame.data[0].marker.color

        fig.data[len(fig.data)-1].x = first_frame.data[1].x
        fig.data[len(fig.data)-1].y = first_frame.data[1].y
        if hasattr(first_frame.layout, 'annotations'):
             fig.layout.annotations = first_frame.layout.annotations
        else:
             fig.layout.annotations = []

    fig.show()

# --- 더미 CSV 데이터 생성 (Dummy CSV Data Generation with varied column names) ---
dummy_csv_data_ahu1_oac = """Timestamp,AHU01_PH/C_온도_입구,AHU01_PH/C_습도_입구,AHU01_PC/C_Zone1_온도,AHU01_PC/C_Zone1_습도,AHU01_CoolingCoil_Air_온도_Avg,AHU01_CoolingCoil_Air_습도_Avg,AHU01_HC_승온후_온도값,AHU01_HC_승온후_습도값
2023-07-15 10:00:00,30.0,65,27.0,75,16.0,92,21.0,68
2023-07-15 10:10:00,30.2,66,26.5,76,14.0,96,23.0,58
2023-07-15 10:20:00,30.5,64,26.0,77,12.0,99,25.0,48
"""

dummy_csv_data_ahu2_oac_control = """Timestamp,ZoneA_PREHEAT_온도,ZoneA_PREHEAT_습도,ZoneA_PRECOOL_온도,ZoneA_PRECOOL_습도,MainUnit_C/C_온도_출구,MainUnit_C/C_습도_출구,PostZone_HEAT_온도,PostZone_HEAT_습도
2023-07-16 14:00:00,28.0,70,25.0,80,17.0,88,20.0,65
2023-07-16 14:10:00,28.2,72,24.5,81,16.0,93,22.5,55
2023-07-16 14:20:00,28.5,71,24.0,82,15.0,97,24.5,45
"""
dummy_csv_data_no_oac = """Timestamp,Sensor_Temp,Sensor_RH
2023-07-17 09:00:00,22,55
"""

# Simulated file system
VIRTUAL_FILE_SYSTEM = {
    "data_ahu1_oac.csv": dummy_csv_data_ahu1_oac,
    "data_ahu2_oac_control.csv": dummy_csv_data_ahu2_oac_control,
    "other_sensor_data.csv": dummy_csv_data_no_oac,
    "report_oac_summary.txt": "This is not a csv"
}

# --- 실행 (Execution) ---
if __name__ == '__main__':
    # 1. Find and select OAC CSV file (simulated)
    available_oac_files = find_oac_csv_files_simulated(VIRTUAL_FILE_SYSTEM)
    selected_filename, selected_csv_content = select_csv_file_simulated(available_oac_files)

    if selected_csv_content:
        print(f"\nLoading data from simulated file: {selected_filename}")
        try:
            # Load data into DataFrame
            df_loaded = pd.read_csv(io.StringIO(selected_csv_content))
            if 'Timestamp' not in df_loaded.columns:
                raise ValueError("CSV must contain a 'Timestamp' column.")
            df_loaded['Timestamp'] = pd.to_datetime(df_loaded['Timestamp'])

            # 2. Dynamically map columns
            mapped_stages = map_columns_from_csv(df_loaded.columns)

            if mapped_stages:
                 # 3. Create interactive chart
                create_interactive_psychrometric_chart_plotly(df_loaded, mapped_stages, selected_filename)
            else:
                print("\nCould not proceed to chart generation due to column mapping issues.")

        except Exception as e:
            print(f"An error occurred processing the file {selected_filename}: {e}")
    else:
        print("No file selected or an error occurred. Exiting.")

    # --- How to use with a real file system (example) ---
    # DATA_DIRECTORY = "./my_csv_data/" # Define your data directory
    #
    # # 1. Find actual OAC CSV files
    # actual_oac_files = []
    # if os.path.exists(DATA_DIRECTORY):
    #     for filename in os.listdir(DATA_DIRECTORY):
    #         if "oac" in filename.lower() and filename.endswith(".csv"):
    #             actual_oac_files.append(os.path.join(DATA_DIRECTORY, filename))
    #
    # if not actual_oac_files:
    #     print(f"No 'OAC' CSV files found in directory: {DATA_DIRECTORY}")
    # else:
    #     print("\nAvailable 'OAC' CSV files:")
    #     for i, filepath in enumerate(actual_oac_files):
    #         print(f"  {i+1}. {os.path.basename(filepath)}")
    #
    #     while True:
    #         try:
    #             choice = int(input(f"Select a file by number (1-{len(actual_oac_files)}): "))
    #             if 1 <= choice <= len(actual_oac_files):
    #                 selected_filepath = actual_oac_files[choice-1]
    #                 break
    #             else:
    #                 print("Invalid choice.")
    #         except ValueError:
    #             print("Invalid input.")
    #
    #     print(f"\nLoading data from: {selected_filepath}")
    #     try:
    #         df_loaded = pd.read_csv(selected_filepath)
    #         if 'Timestamp' not in df_loaded.columns:
    #             raise ValueError("CSV must contain a 'Timestamp' column.")
    #         df_loaded['Timestamp'] = pd.to_datetime(df_loaded['Timestamp'])
    #         mapped_stages = map_columns_from_csv(df_loaded.columns)
    #         if mapped_stages:
    #             create_interactive_psychrometric_chart_plotly(df_loaded, mapped_stages, os.path.basename(selected_filepath))
    #     except Exception as e:
    #         print(f"An error occurred: {e}")

Simulating file search...

Available 'OAC' CSV files (simulated):
  1. data_ahu1_oac.csv
  2. data_ahu2_oac_control.csv
